In [1]:
# Etapas

# 1. Importar bibliotecas
# 2. Importar a listagem de produtos a serem repostos
# 3. Abrir a cotação para a cote fácil
# 4. Anexar o arquivo de produtos a serem repostos e gerar a cotação

In [2]:
# 1. Importar bibliotecas
import pandas as pd
import autoit
from functions import *
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pyautogui
import time
from pynput.keyboard import Key, Controller
from datetime import datetime
import warnings
#warnings.filterwarnings("ignore")
options = webdriver.ChromeOptions()
options.add_argument("--headless")

link_ip = '179.187.90.77'
from password import passwordTrier, passwordCoteFacil
login_trier = passwordTrier()[0]
password_trier = passwordTrier()[1]
login_cotefacil = passwordCoteFacil()[0]
password_cotefacil = passwordCoteFacil()[1]


In [3]:
# 2. Importar a listagem de produtos a serem repostos
def exporta_lista_rep_auto():
    driver = webdriver.Chrome()
    #driver.implicitly_wait(30)
    driver.get(f"http://{link_ip}:4647/sgfpod1/Login.pod")
    sleep(1)
    driver.find_element(By.NAME, 'id_cod_usuario').send_keys(login_trier)
    sleep(1)
    driver.find_element(By.NAME, 'nom_senha').send_keys(password_trier)
    sleep(1)
    driver.find_element(By.NAME, 'login').click()
    sleep(1)
    driver.get(f"http://{link_ip}:4647/web-drogaria-app/#/compras/painel-indicativo")
    sleep(1)
    driver.get(f"http://{link_ip}:4647/web-drogaria-app/#/compras/dose-certa-novo/0?profile=139")
    sleep(1)
    
    # Aguarde até que os elementos estejam presentes na página
    wait = WebDriverWait(driver, 10)
    elemento_inicial = wait.until(EC.presence_of_element_located((By.XPATH, "//li[contains(@class, 'cdk-drop-list')]//button[contains(@class, 'arrastar-todos') and contains(., 'Arraste para comprar todos')]")))
    elemento_final = wait.until(EC.presence_of_element_located((By.XPATH, "//p[contains(@class, 'area-arrastar')]")))

    print(elemento_inicial.text)
    if elemento_inicial.text == 'Arraste para comprar todos (0)':
        print('Nenhum produto para reposição automática')
    
    else:
        # Clique e arraste o elemento de um ponto a outro
        action_chains = ActionChains(driver)
        action_chains.drag_and_drop(elemento_inicial, elemento_final).perform()
        
        # Aguarde até que o botão "Sim" esteja presente na página
        botao_sim = wait.until(EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'mat-flat-button') and contains(., 'Sim')]")))

        # Clique no botão "Sim"
        botao_sim.click()
        
        # Aguarde até que o campo de input esteja presente na página
        campo_input = wait.until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'ng-input')]//input[@role='combobox']")))

        # Digite o código "309" no campo de input e pressione Enter
        campo_input.send_keys("309")
        
        # Aguarde até que o elemento "309 - COTE FACIL" esteja presente na página
        elemento_cote_facil = wait.until(EC.presence_of_element_located((By.XPATH, "//span[contains(@class, 'ng-option-label') and contains(., '309 - COTE FACIL')]")))

        # Clique no elemento "309 - COTE FACIL"
        elemento_cote_facil.click()
        
        # Pressione a tecla "TAB" usando o PyAutoGUI
        pyautogui.press('tab')
        
        # Pressione a tecla "ENTER" usando o PyAutoGUI
        pyautogui.press('enter')

        # Aguarde até que o botão "Gerar pedidos" esteja presente na página
        botao_gerar_pedidos = wait.until(EC.presence_of_element_located((By.XPATH, "//span[contains(@class, 'mat-button-wrapper') and contains(., 'Gerar pedidos')]")))

        # Clique no botão "Gerar pedidos"
        sleep(5)
        botao_gerar_pedidos.click()
        
        # Aguarde até que o botão "Gerar todos os pedidos" esteja presente na página
        botao_gerar_todos_pedidos = wait.until(EC.presence_of_element_located((By.XPATH, "//span[contains(@class, 'mat-menu-item') and contains(b/text(), 'Gerar todos os pedidos')]")))

        # Clique no botão "Gerar todos os pedidos"
        botao_gerar_todos_pedidos.click()
        
        # Aguarde até que o campo de usuário esteja presente na página
        campo_usuario = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "mat-input-3")))

        # Preencha o campo de usuário
        campo_usuario.send_keys(login_trier)
        
        # Aguarde até que o campo de senha esteja presente na página
        campo_senha = driver.find_element(By.ID, "mat-input-4")

        # Preencha o campo de senha
        campo_senha.send_keys(password_trier)

        # Localize o botão "Autenticar" e clique nele
        botao_autenticar = driver.find_element(By.XPATH, "//button[contains(@class, 'mat-flat-button') and contains(., 'Autenticar')]")
        botao_autenticar.click()
        
    # ir para a pagina de gerenciamento de pedidos
    driver.get(f"http://{link_ip}:4647/web-drogaria-app/#/compras/gerenciamento-pedido")
    
    sleep(10)  # Adicione um tempo de espera para garantir que os pedidos sejam carregados na tabela
    
    # Obter a data de hoje no formato "dd/mm/yyyy"
    data_hoje = datetime.now().strftime('%d/%m/%Y')
    
    # Obter o código HTML da página
    html = driver.page_source
    
    # Parsear o código HTML com Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    
    # Encontrar a tabela de pedidos
    tabela_pedidos = soup.find('div', class_='ag-center-cols-container')
    
    # Inicializar uma lista para armazenar os pedidos de hoje
    pedidos_hoje = []
    
    # Iterar pelas linhas da tabela de pedidos e encontrar os pedidos criados hoje
    if tabela_pedidos:
        for row in tabela_pedidos.find_all('div', role='row'):
            cells = row.find_all('div', role='gridcell')
            if len(cells) >= 5:  # Garantir que a linha tenha informações suficientes
                datEmissao = cells[4].get('title', '')
                if datEmissao == data_hoje:
                    numero_pedido = cells[1].get('title', '')
                    pedidos_hoje.append(numero_pedido)
    
    # Obter o número de pedidos criados hoje
    numero_pedidos_hoje = len(pedidos_hoje)
    
    print(f"Número de pedidos criados hoje: {numero_pedidos_hoje}")
    print("Números dos pedidos criados hoje:", pedidos_hoje)
    sleep(2)

    # Clicar no primeiro pedido na coluna "Cotação"
    try:
        # Localizar o ícone de cotação para o primeiro pedido
        xpath_cotacao = "//div[@col-id='cotacao']//i[contains(@class, 'fas fa-calculator')]"
        botao_cotacao = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, xpath_cotacao))
        )
        
        # Role a página para que o elemento fique visível
        driver.execute_script("arguments[0].scrollIntoView();", botao_cotacao)
        
        # Clique no ícone de cotação
        botao_cotacao.click()
        print("Clicou no primeiro pedido na opção cotação.")
    except:
        print("Não foi possível clicar no primeiro pedido na opção cotação.")



    # Clicar no elemento <img src="./assets/icons/cotacao/sicfebrafar-logo.png" style="height: 16px;">
    try:
        xpath_logo = "//img[contains(@src, './assets/icons/cotacao/sicfebrafar-logo.png')]"
        botao_logo = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, xpath_logo))
        )
        
        # Role a página para que o elemento fique visível
        driver.execute_script("arguments[0].scrollIntoView();", botao_logo)
        
        # Clique no elemento de logo
        botao_logo.click()
        print("Clicou no elemento de logo.")
    except:
        print("Não foi possível clicar no elemento de logo.")

    # Esperar um tempo para o download ser concluído
    sleep(5)
    
    # Esperar o download ser concluído
    downloads_directory = "C:\\Users\\Samsung\\Downloads"  # Caminho para a pasta de downloads
    timeout = 60  # Defina o tempo máximo de espera em segundos
    start_time = time.time()
    latest_file = None
    
    while time.time() - start_time < timeout and latest_file is None:
        list_of_files = os.listdir(downloads_directory)
        if list_of_files:
            latest_file = max(list_of_files, key=lambda x: os.path.getctime(os.path.join(downloads_directory, x)))
            file_path = os.path.join(downloads_directory, latest_file)
            print("Caminho do arquivo baixado:", file_path)
            break
        time.sleep(1)
    
    if latest_file is None:
        print("O arquivo não foi encontrado na pasta de downloads após o tempo limite.")
    
    return file_path
    
    

In [4]:
file_path = exporta_lista_rep_auto()

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 113
Current browser version is 115.0.5790.110 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00518893+48451]
	(No symbol) [0x004AB8A1]
	(No symbol) [0x003B5058]
	(No symbol) [0x003D5D9C]
	(No symbol) [0x003D1AA3]
	(No symbol) [0x003D0019]
	(No symbol) [0x00400798]
	(No symbol) [0x0040047C]
	(No symbol) [0x003FA0B6]
	(No symbol) [0x003D7E08]
	(No symbol) [0x003D8F2D]
	GetHandleVerifier [0x00778E3A+2540266]
	GetHandleVerifier [0x007B8959+2801161]
	GetHandleVerifier [0x007B295C+2776588]
	GetHandleVerifier [0x005A2280+612144]
	(No symbol) [0x004B4F6C]
	(No symbol) [0x004B11D8]
	(No symbol) [0x004B12BB]
	(No symbol) [0x004A4857]
	BaseThreadInitThunk [0x76447D59+25]
	RtlInitializeExceptionChain [0x76F3B74B+107]
	RtlClearBits [0x76F3B6CF+191]


In [ ]:
def upload_cotefacil(file_path):
    driver = webdriver.Chrome()
    driver.get(f"https://sistemas.cotefacil.com/CTFLLogan-webapp/login.jsf")

    # Aguardar até que o campo de usuário esteja presente na página
    wait = WebDriverWait(driver, 10)
    username_field = wait.until(EC.presence_of_element_located((By.ID, "frmLogin:username")))
    
    # Aguardar até que o campo de senha esteja presente na página
    password_field = wait.until(EC.presence_of_element_located((By.ID, "frmLogin:password")))

    # Preencher o campo de usuário e senha
    username = login_cotefacil  
    password = password_cotefacil
    username_field.send_keys(username)
    password_field.send_keys(password)

    # Aguardar até que o botão "Entrar" esteja clicável na página
    login_button = wait.until(EC.element_to_be_clickable((By.ID, "frmLogin:loginButton")))

    # Clicar no botão "Entrar" para efetuar o login
    login_button.click()
    sleep(5)
    
    driver.get(f"https://sistemas.cotefacil.com/CTFLLogan-webapp/spring/pages/comprador/cotacoes?execution=e2s1")
    
    # Aguardar até que o elemento checkbox esteja presente na página usando atributo "name"
    checkbox_element = wait.until(EC.presence_of_element_located((By.NAME, "novaCotacao:filiais:4:j_id748")))

    # Clicar no elemento checkbox
    checkbox_element.click()
    
    # Aguardar até que o elemento checkbox esteja presente na página usando atributo "name"
    checkbox_element = wait.until(EC.presence_of_element_located((By.NAME, "novaCotacao:fonecedores:1:j_id670")))

    # Desmarcar o elemento checkbox (se já estiver marcado)
    if checkbox_element.is_selected():
        checkbox_element.click()
    
    # Obter a data e hora atual
    data_hora_atual = datetime.now()
    
    # Formatar a data e hora no formato desejado (dd_mm_YYYY_HH_MM_SS)
    data_hora_formatada = data_hora_atual.strftime("%d%m%Y_%H%M")
    
    nome_filial = "VARGEM"
    
    # Concatenar o nome da filial, data e hora para formar o nome da cotação
    nome_cotacao = f"*{nome_filial}_{data_hora_formatada}_REP_AUTO"
    
    # Localizar o campo de texto para o nome da cotação pelo id
    campo_nome_cotacao = driver.find_element(By.ID, "novaCotacao:titulo")
    
    # Preencher o nome da cotação no campo de texto
    campo_nome_cotacao.send_keys(nome_cotacao)
    

    # Digitar a data atual diretamente no campo
    data_atual = datetime.now().strftime("%d/%m/%Y")

    # Localizar o campo da data pelo id
    campo_data = driver.find_element(By.ID, "novaCotacao:vencimentoInputDate")

    # Clicar no campo da data para ativar o calendário
    campo_data.click()
    
    # Localizar o elemento do dia atual (hoje) no calendário e clicar nele diretamente
    dia_atual_element = driver.find_element(By.XPATH, "//td[contains(@class, 'rich-calendar-today')]")
    dia_atual_element.click()

    # Localizar o campo de hora pelo id
    campo_hora = driver.find_element(By.ID, "novaCotacao:hora")

    # Preencher o campo de hora com "22:00"
    campo_hora.send_keys("22:00")
    
    # Localizar o campo de seleção pelo id
    campo_selecao = driver.find_element(By.ID, "novaCotacao:condicaoPagamento")

    # Utilizar a classe Select para selecionar a opção "A prazo" pelo texto visível
    select = Select(campo_selecao)
    select.select_by_visible_text("A prazo")
    
    # Localizar o elemento de rádio pelo id e clicar nele
    elemento_radio = driver.find_element(By.ID, "novaCotacao:tipoEntrega:0")
    elemento_radio.click()

    sleep(2)
    
    # Localizar o botão "Criar Cotação" pelo id e clicar nele
    botao_criar_cotacao = driver.find_element(By.ID, "novaCotacao:j_id756")
    botao_criar_cotacao.click()
    
    sleep(1)
    
    # Localizar o elemento radio pelo valor "false" e clicar nele
    elemento_radio = driver.find_element(By.XPATH, "//input[@type='radio' and @name='novaCotacao:j_id795' and @value='false']")
    elemento_radio.click()
    
    sleep(3)

    # Clicar no elemento para abrir a caixa de seleção de arquivo
    caminho_do_arquivo = f"{file_path}"

    elemento_upload = driver.find_element(By.CLASS_NAME, "rich-fileupload-button-border")
    elemento_upload.click()
    sleep(3)
    
    keyboard = Controller()
    keyboard.type(caminho_do_arquivo)
    keyboard.press(Key.enter)
    keyboard.release(Key.enter)
    
    sleep(10)

    elemento_encerrar = driver.find_element(By.ID, "novaCotacao:j_id1109")
    elemento_encerrar.click()
    
    sleep(5)
    
    elemento_enviar = driver.find_element(By.ID, 'novaCotacao:j_id1305')
    elemento_enviar.click()
    

    sleep(30)

In [ ]:
upload_cotefacil(file_path)

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 113
Current browser version is 115.0.5790.110 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00518893+48451]
	(No symbol) [0x004AB8A1]
	(No symbol) [0x003B5058]
	(No symbol) [0x003D5D9C]
	(No symbol) [0x003D1AA3]
	(No symbol) [0x003D0019]
	(No symbol) [0x00400798]
	(No symbol) [0x0040047C]
	(No symbol) [0x003FA0B6]
	(No symbol) [0x003D7E08]
	(No symbol) [0x003D8F2D]
	GetHandleVerifier [0x00778E3A+2540266]
	GetHandleVerifier [0x007B8959+2801161]
	GetHandleVerifier [0x007B295C+2776588]
	GetHandleVerifier [0x005A2280+612144]
	(No symbol) [0x004B4F6C]
	(No symbol) [0x004B11D8]
	(No symbol) [0x004B12BB]
	(No symbol) [0x004A4857]
	BaseThreadInitThunk [0x76447D59+25]
	RtlInitializeExceptionChain [0x76F3B74B+107]
	RtlClearBits [0x76F3B6CF+191]


In [ ]:
file_path

'C:\\Users\\Samsung\\Downloads\\SICF_20230725125526.txt'